In [6]:
%pip install langchain sentence-transformers pypdf chromadb
%pip install -U langchain-ollama langchain_community langchain_huggingface


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Note: you may need to restart the kernel to use updated packages.


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Note: you may need to restart the kernel to use updated packages.


In [ ]:
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import CharacterTextSplitter
# from langchain.embeddings import HuggingFaceEmbeddings
from langchain_huggingface import HuggingFaceEmbeddings
from sentence_transformers import SentenceTransformer
from langchain.vectorstores import Chroma
from langchain_ollama import OllamaLLM
from langchain.chains import RetrievalQA

filename = "./doc.pdf"
loader = PyPDFLoader(filename)
documents = loader.load()

text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=30, separator="\n")
docs = text_splitter.split_documents(documents=documents)


embedding_model_name = "sentence-transformers/all-mpnet-base-v2"
model_kwargs = {"device": "cpu"}
embeddings = HuggingFaceEmbeddings(
    model_name=embedding_model_name, model_kwargs=model_kwargs
)

vectorstore = Chroma.from_documents(
    docs, embeddings, persist_directory="./chroma_store_1"
)
llm = OllamaLLM(model="llama3", base_url="http://llm:11434")
retriever = vectorstore.as_retriever()
qa = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=retriever)

while True:
    query = input("Type your query (or type 'Exit' to quit): \n")
    if query.lower() == "exit":
        break
    result = qa.run(query)
    print(result)


Type your query (or type 'Exit' to quit): 
 give me an example of one-shot prompt


According to the context, a one-shot prompt provides a single example for the model to imitate. Therefore, an example of a one-shot prompt could be:

"Write a short story about a person who discovers they have superpowers."

This prompt gives the model a single example (a concept or idea) to follow and generate text based on.


Type your query (or type 'Exit' to quit): 
 give me the shortest and elegant zero shot prompt


Based on the provided context, a zero-shot prompt is a simple description of a task with no additional examples. Given that, I would suggest:

"What is [X]?"

This prompt provides a clear task description (what something is) and leaves it up to the LLM to generate an answer. The "[X]" placeholder can be replaced with any topic or concept you'd like the LLM to generate text about.

Note: This is just one example of a zero-shot prompt, and there are many other possibilities depending on the specific context and desired output.
